In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Vamos a ver un ejemplo con un conjunto de opiniones (positivas o negativas) de restaurantes del recomendador Yelp

import pandas as pd

yelp_file = '/content/drive/MyDrive/Colab Notebooks/IA2/yelp_labelled.txt'

df=pd.read_csv(yelp_file, delimiter='\t', names=['sentence', 'sentiment'])

df

,sentence,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


# Usando word embeddings

Este notebook está basado en el notebook del segundo ejemplo del Capítulo 6, Sección 1 de [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff).

[Aquí hay una versión gratuita del capítulo](https://freecontent.manning.com/deep-learning-for-text/)

---


Las word-embeddings son representaciones de palabras en espacios de varias dimensiones en los que cada palabra del diccionario es representada mediante un vector de varias dimensiones (decenas o centenas) que permiten capturar su relación con otras palabras

Existen distintas técnicas para estimar word-embeddings a partir de un corpus de documentos y fijando el número de dimensiones que queremos usar.

El resultado es una matriz de tantas columnas como palabras tenga el diccionario y tantas filas como dimensiones hayamos determinado.


There are two ways to obtain word embeddings:

* Learn word embeddings jointly with the main task you care about (e.g. document classification or sentiment prediction).
In this setup, you would start with random word vectors, then learn your word vectors in the same way that you learn the weights of a neural network.
* Load into your model word embeddings that were pre-computed using a different machine learning task than the one you are trying to solve.
These are called "pre-trained word embeddings".

Let's take a look at both.

## Learning word embeddings with the `Embedding` layer


Is there some "ideal" word embedding space that would perfectly map human language and could be used for any natural language processing
task? Possibly, but in any case, we have yet to compute anything of the sort. Also, there isn't such a thing as "human language", there are
many different languages and they are not isomorphic, as a language is the reflection of a specific culture and a specific context. But more
pragmatically, what makes a good word embedding space depends heavily on your task: the perfect word embedding space for an
English-language movie review sentiment analysis model may look very different from the perfect embedding space for an English-language
legal document classification model, because the importance of certain semantic relationships varies from task to task.

It is thus reasonable to __learn__ a new embedding space with every new task. Thankfully, backpropagation makes this really easy, and Keras makes it
even easier. It's just about learning the weights of a layer: the `Embedding` layer.


The `Embedding` layer is best understood as a dictionary mapping integer indices (which stand for specific words) to dense vectors. It takes
as input integers, it looks up these integers into an internal dictionary, and it returns the associated vectors. It's effectively a dictionary lookup.


The `Embedding` layer takes as input a 2D tensor of integers, of shape `(samples, sequence_length)`, where each entry is a sequence of
integers. It can embed sequences of variable lengths, so for instance we could feed into our embedding layer above batches that could have
shapes `(32, 10)` (batch of 32 sequences of length 10) or `(64, 15)` (batch of 64 sequences of length 15). All sequences in a batch must
have the same length, though (since we need to pack them into a single tensor), so sequences that are shorter than others should be padded
with zeros, and sequences that are longer should be truncated.

When you instantiate an `Embedding` layer, its weights (its internal dictionary of token vectors) are initially random, just like with any
other layer. During training, these word vectors will be gradually adjusted via backpropagation, structuring the space into something that the
downstream model can exploit. Once fully trained, your embedding space will show a lot of structure -- a kind of structure specialized for
the specific problem you were training your model for.


In [2]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

max_words = 1500
max_comment_length = 20

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df.sentence)

sequences = tokenizer.texts_to_sequences(df.sentence)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
max_words = len(word_index)

data = pad_sequences(sequences, maxlen=max_comment_length)

ModuleNotFoundError: No module named 'keras.preprocessing.text'

In [ ]:
print(df.sentence[4])
print(data[4])
print(df.sentiment[4])

The selection on the menu was great and so were the prices.
[  0   0   0   0   0   0   0   0   1 166  35   1 109   4  22   2  26  27
   1 167]
1


In [ ]:
from sklearn.model_selection import train_test_split

#valor de random state
rs=333

d=df.values

x_train, x_test, y_train, y_test = train_test_split(data, df.sentiment, test_size=0.20, random_state=rs, stratify = df.sentiment)

print("Training texts:", len(y_train))
print("Test texts:", len(y_test))

Training texts: 800
Test texts: 200


In [ ]:
# Fijamos el tamaño de los embedding a 50 dimensiones

embedding_dim = 50

In [ ]:
# MODELO 1. SIN EMBEDDINGS PRE-ENTRENADOS

from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model1 = Sequential()
# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs


model1.add(Embedding(max_words, embedding_dim, input_length=max_comment_length))
# After the Embedding layer, our activations have shape `(max_words, max_comment_length, embedding_dim)`.

# We flatten the 3D tensor of embeddings into a 2D tensor of shape `(max_words, max_comment_length * embedding_dim)`

model1.add(Flatten())

# We add the classifier on top
model1.add(Dense(1, activation='sigmoid'))

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.summary()

history = model1.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_test, y_test))

score1 = model1.evaluate(x_test, y_test)

print("Accuracy: %.2f%%" % (score1[1]*100))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 50)            103550    
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 1)                 1001      
                                                                 
Total params: 104551 (408.40 KB)
Trainable params: 104551 (408.40 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
25/25 [==============================] - 1s 15ms/step - loss: 0.6930 - accuracy: 0.5150 - val_loss: 0.6893 - val_accuracy: 0.5400
Epoch 2/20
25/25 [==============================] - 0s 5ms/step - loss: 0.6706 - accuracy: 0.6550 - val_loss: 0.6864 - val_accuracy: 0.5550

Utilizando una capa de embedding directamente obtenemos un 69%.

## Using pre-trained word embeddings


Sometimes, you have so little training data available that could never use your data alone to learn an appropriate task-specific embedding
of your vocabulary. What to do then?

Instead of learning word embeddings jointly with the problem you want to solve, you could be loading embedding vectors from a pre-computed
embedding space known to be highly structured and to exhibit useful properties -- that captures generic aspects of language structure.

Such word embeddings are generally computed using word occurrence statistics (observations about what words co-occur in sentences or
documents), using a variety of techniques, some involving neural networks, others not. The idea of a dense, low-dimensional embedding space
for words, computed in an unsupervised way, was initially explored by Bengio et al. in the early 2000s, but it only started really taking
off in research and industry applications after the release of one of the most famous and successful word embedding scheme: the Word2Vec
algorithm, developed by Mikolov at Google in 2013. Word2Vec dimensions capture specific semantic properties, e.g. gender.

There are various pre-computed databases of word embeddings that can download and start using in a Keras `Embedding` layer. Word2Vec is one
of them. Another popular one is called "GloVe", developed by Stanford researchers in 2014. It stands for "Global Vectors for Word
Representation", and it is an embedding technique based on factorizing a matrix of word co-occurrence statistics. Its developers have made
available pre-computed embeddings for millions of English tokens, obtained from Wikipedia data or from Common Crawl data.

Let's take a look at how you can get started using GloVe embeddings in a Keras model. The same method will of course be valid for Word2Vec
embeddings or any other word embedding database that you can download.

### Download the GloVe word embeddings


Head to `https://nlp.stanford.edu/projects/glove/` (where you can learn more about the GloVe algorithm), and download the pre-computed
embeddings from 2014 English Wikipedia + Gigaword 5. There are several word embedding files. We will use the small word embedding with 50 dimensions (glove.6B.50d).

### Pre-process the embeddings


Let's parse the un-zipped file (it's a `txt` file) to build an index mapping words (as strings) to their vector representation (as number
vectors).

In [ ]:
import os
import numpy as np

glove_dir = '/content/drive/MyDrive/Colab Notebooks/IA2'

embeddings_index = {}
f = open(os.path.join('./', 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.



Now let's build an embedding matrix that we will be able to load into an `Embedding` layer. It must be a matrix of shape `(max_words,
embedding_dim)`, where each entry `i` contains the `embedding_dim`-dimensional vector for the word of index `i` in our reference word index
(built during tokenization). Note that the index `0` is not supposed to stand for any word or token -- it's a placeholder.

In [ ]:
embedding_dim = 50
max_words = 1500

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

embedding_matrix

NameError: name 'word_index' is not defined

### Define a model

We will be using the same model architecture as before:

In [ ]:
# MODELO 2. EMBEDDINGS PRE-ENTRENADOS CONGELADOS

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model2 = Sequential()
model2.add(Embedding(max_words, embedding_dim, input_length=max_comment_length))
model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 50)            103550    
                                                                 
 flatten_1 (Flatten)         (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 104551 (408.40 KB)
Trainable params: 104551 (408.40 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Load the GloVe embeddings in the model


The `Embedding` layer has a single weight matrix: a 2D float matrix where each entry `i` is the word vector meant to be associated with
index `i`. Simple enough. Let's just load the GloVe matrix we prepared into our `Embedding` layer, the first layer in our model:

In [ ]:
model2.layers[0].set_weights([embedding_matrix])
model2.layers[0].trainable = False


Additionally, we freeze the embedding layer (we set its `trainable` attribute to `False`), then one part of a model is pre-trained (like our `Embedding` layer), and parts are randomly initialized (like our classifier), the pre-trained parts should not be updated during training to avoid forgetting what they already know.

### Train and evaluate

Let's compile our model and train it:

In [ ]:
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model2.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_test, y_test))

score2 = model2.evaluate(x_test, y_test)

Epoch 1/20
25/25 [==============================] - 1s 13ms/step - loss: 0.7239 - accuracy: 0.5250 - val_loss: 0.6925 - val_accuracy: 0.5750
Epoch 2/20
25/25 [==============================] - 0s 5ms/step - loss: 0.6596 - accuracy: 0.6000 - val_loss: 0.6696 - val_accuracy: 0.6000
Epoch 3/20
25/25 [==============================] - 0s 5ms/step - loss: 0.6193 - accuracy: 0.6687 - val_loss: 0.6516 - val_accuracy: 0.6050
Epoch 4/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5886 - accuracy: 0.7013 - val_loss: 0.6432 - val_accuracy: 0.6400
Epoch 5/20
25/25 [==============================] - 0s 5ms/step - loss: 0.5628 - accuracy: 0.7262 - val_loss: 0.6347 - val_accuracy: 0.6650
Epoch 6/20
25/25 [==============================] - 0s 5ms/step - loss: 0.5405 - accuracy: 0.7525 - val_loss: 0.6254 - val_accuracy: 0.6500
Epoch 7/20
25/25 [==============================] - 0s 5ms/step - loss: 0.5230 - accuracy: 0.7600 - val_loss: 0.6206 - val_accuracy: 0.6700
Epoch 8/20
25/25 [=


We obtain 70% accuracy with the pre-trained model. It's a better result because we don't have too much training data.



We can also try to train the same model without freezing the embedding layer. In that case, we would be adapting our word embedding learning a classification task.

Let's try it:

In [ ]:
# MODELO3. EMBEDDINGS PREENTRENADOS SIN CONGELAR

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model3 = Sequential()
model3.add(Embedding(max_words, embedding_dim, input_length=max_comment_length))
model3.add(Flatten())
model3.add(Dense(1, activation='sigmoid'))
model3.summary()

model3.layers[0].set_weights([embedding_matrix])
model3.layers[0].trainable = True

model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model3.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_test, y_test))

score3 = model3.evaluate(x_test, y_test)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 50)            103550    
                                                                 
 flatten_2 (Flatten)         (None, 1000)              0         
                                                                 
 dense_2 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 104551 (408.40 KB)
Trainable params: 104551 (408.40 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
25/25 [==============================] - 1s 13ms/step - loss: 0.7162 - accuracy: 0.5500 - val_loss: 0.7171 - val_accuracy: 0.5550
Epoch 2/20
25/25 [==============================] - 0s 5ms/step - loss: 0.6322 - accuracy: 0.6587 - val_loss: 0.6899 - val_accuracy: 0.63


We obtain 75% accuracy with the pre-trained model not freezed. In this case, it's the best configuration to our classification problem.


# Resumen de resultados

In [ ]:
print("Sin word embeddings pre-entrenados")
print("Accuracy: %.2f%%" % (score1[1]*100))
print("Con word embeddings pre-entrenados congelados")
print("Accuracy: %.2f%%" % (score2[1]*100))
print("Con word embeddings pre-entrenados sin congelar")
print("Accuracy: %.2f%%" % (score3[1]*100))

Sin word embeddings pre-entrenados
Accuracy: 70.00%
Con word embeddings pre-entrenados congelados
Accuracy: 71.00%
Con word embeddings pre-entrenados sin congelar
Accuracy: 74.00%
